In [11]:
import pandas as pd
import os
import random
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill
from datetime import datetime
from copy import copy

# === Prompt for main folder path ===
main_folder = input("\U0001F4C1 Enter the path to the main folder containing 'input' and 'output' subfolders: ").strip()

input_folder = os.path.join(main_folder, 'input')
output_folder = os.path.join(main_folder, 'output')

if not os.path.isdir(input_folder) or not os.path.isdir(output_folder):
    print("❌ Ensure 'input' and 'output' folders exist inside the specified path.")
    exit()

valuation_path = os.path.join(input_folder, 'Annuity valuation.xlsx')
register_path = os.path.join(input_folder, 'Annuity register.xlsx')

if not os.path.isfile(valuation_path) or not os.path.isfile(register_path):
    print("❌ Input files not found in 'input' folder.")
    exit()

# === Ask for valuation date ===
input_date = input("\U0001F4C5 Enter the valuation date (DD/MM/YYYY): ")
try:
    valuation_date = pd.to_datetime(input_date, format='%d/%m/%Y')
except:
    print("❌ Invalid date format.")
    exit()

# === Load register and filter by date ===
try:
    register_df = pd.read_excel(register_path, dtype=str)
    register_df.columns = [col.strip() for col in register_df.columns]
    register_df['Valuation date'] = pd.to_datetime(register_df['Valuation date'], errors='coerce')
    filtered_df = register_df[register_df['Valuation date'] == valuation_date].copy()
    if filtered_df.empty:
        print(f"⚠️ No records found for {valuation_date.date()}.")
        exit()
except Exception as e:
    print(f"❌ Error processing register file: {e}")
    exit()

# === Clean whitespaces and non-printables ===
for col in ['POLICY NUMBER', 'NAME OF ANNUITANT', 'SOURCE OF FUNDS']:
    if col in filtered_df.columns:
        filtered_df[col] = filtered_df[col].astype(str).str.strip().str.replace(r'[^\x20-\x7E]', '', regex=True)

# === Load workbook and target sheet ===
wb = load_workbook(valuation_path)
target_sheet = "Current Valuation Data"
backup_sheet = "Previous Valuation Data"

if target_sheet not in wb.sheetnames:
    print(f"❌ Sheet '{target_sheet}' not found.")
    exit()

ws = wb[target_sheet]

# === Read header row ===
header_row = [cell.value.strip() for cell in ws[1] if cell.value is not None]

# === Align Data ===
filtered_df.columns = [c.strip() for c in filtered_df.columns]
aligned_df = pd.DataFrame(columns=header_row)
for col in header_row:
    match_col = next((c for c in filtered_df.columns if c.strip().lower() == col.strip().lower()), None)
    aligned_df[col] = filtered_df[match_col] if match_col else ""

# === Replace TBA in POLICY NUMBER ===
if 'POLICY NUMBER' in aligned_df.columns:
    base_suffix = valuation_date.strftime("%b%y")
    tba_counter = [1]
    def assign_tba_id(val, counter):
        if isinstance(val, str) and val.strip().upper() == 'TBA':
            unique_id = f"TBA{counter[0]}{base_suffix}"
            counter[0] += 1
            return unique_id
        return val
    aligned_df['POLICY NUMBER'] = aligned_df['POLICY NUMBER'].apply(assign_tba_id, counter=tba_counter)

# === Format SPOUSE REVERSION RATE ===
if 'SPOUSE REVERSION RATE' in aligned_df.columns:
    def format_spouse(val):
        try:
            # Convert to float, then to integer
            num = float(val)
            return int(round(num))  # Return as integer for internal use
        except:
            return 0  # Default to 0 if the value is not valid
    aligned_df['SPOUSE REVERSION RATE'] = aligned_df['SPOUSE REVERSION RATE'].apply(format_spouse)

# === Format GUARANTEE ===
if 'GUARANTEE' in aligned_df.columns:
    def format_guarantee(val):
        val_str = str(val).strip().upper()
        if val_str in ["", "NAN"]:
            return "0 YEARS"
        if val_str == "#N/A":
            return "N/A"
        try:
            num = float(val_str)
            return f"{int(num)} YEARS"
        except:
            return val
    aligned_df['GUARANTEE'] = aligned_df['GUARANTEE'].apply(format_guarantee)

# === Format ESCALATION RATE ===
if 'ESCALATION RATE' in aligned_df.columns:
    aligned_df['ESCALATION RATE'] = aligned_df['ESCALATION RATE'].fillna("0")

# === Format PURCHASE PRICE ===
if 'PURCHASE PRICE' in aligned_df.columns:
    def format_purchase_price(val):
        try:
            return float(val)
        except:
            return val
    aligned_df['PURCHASE PRICE'] = aligned_df['PURCHASE PRICE'].apply(format_purchase_price)

# === LAST EXPENSE BENEFIT Validation ===
if 'LAST EXPENSE BENEFIT' in aligned_df.columns:
    if aligned_df['LAST EXPENSE BENEFIT'].str.upper().isin(['#N/A', 'N/A']).any():
        raise ValueError("❌ LAST EXPENSE BENEFIT contains #N/A or N/A values. Script terminated.")

# === Backup sheet ===
if backup_sheet in wb.sheetnames:
    del wb[backup_sheet]
backup_ws = wb.copy_worksheet(ws)
backup_ws.title = backup_sheet

# === Get column indexes for formatting ===
column_indexes = {}
for idx, cell in enumerate(ws[1], start=1):
    header = str(cell.value).strip().upper() if cell.value else ""
    if header:
        column_indexes[header] = idx

last_data_row = 1
for row in range(ws.max_row, 1, -1):
    if ws.cell(row=row, column=column_indexes.get("POLICY NUMBER", 1)).value not in (None, ""):
        last_data_row = row
        break

start_row = last_data_row + 1
template_row = ws[last_data_row]

column_formats = {}
for idx, cell in enumerate(template_row, start=1):
    column_formats[idx] = {
        'font': copy(cell.font),
        'fill': copy(cell.fill),
        'border': copy(cell.border),
        'alignment': copy(cell.alignment),
        'number_format': cell.number_format
    }

# === Generate Light Random Color ===
def generate_light_color():
    r = random.randint(200, 255)
    g = random.randint(200, 255)
    b = random.randint(200, 255)
    return '{:02X}{:02X}{:02X}'.format(r, g, b)

light_color_hex = generate_light_color()
policy_fill = PatternFill(start_color=light_color_hex, end_color=light_color_hex, fill_type="solid")

# === Append data to worksheet ===
for row in dataframe_to_rows(aligned_df, index=False, header=False):
    for col_idx, value in enumerate(row, start=1):
        header_cell = ws.cell(row=1, column=col_idx)
        header_upper = str(header_cell.value).strip().upper() if header_cell.value else ""
        
        # Handle dates correctly
        if header_upper in ["DATE OF BIRTH", "COMMENCEMENT DATE", "VALUATION DATE"]:
            try:
                parsed_date = pd.to_datetime(value, errors='coerce')
                value = parsed_date if pd.notnull(parsed_date) else value
            except:
                pass
        
        # Convert to number if it's a numeric field but not a date or POLICY NUMBER
        if header_upper not in ["POLICY NUMBER", "DATE OF BIRTH", "COMMENCEMENT DATE", "VALUATION DATE"]:
            try:
                num = float(value)
                if not pd.isna(num):
                    value = num
            except:
                pass

        # === Format SPOUSE REVERSION RATE ===
if 'SPOUSE REVERSION RATE' in aligned_df.columns:
    def format_spouse(val):
        try:
            # Convert to float, then to integer
            num = float(val)
            return int(round(num))  # Return as integer for internal use
        except:
            return 0  # Default to 0 if the value is not valid
    aligned_df['SPOUSE REVERSION RATE'] = aligned_df['SPOUSE REVERSION RATE'].apply(format_spouse)

# === Append data to worksheet ===
for row in dataframe_to_rows(aligned_df, index=False, header=False):
    for col_idx, value in enumerate(row, start=1):
        header_cell = ws.cell(row=1, column=col_idx)
        header_upper = str(header_cell.value).strip().upper() if header_cell.value else ""
        
        # Handle dates correctly
        if header_upper in ["DATE OF BIRTH", "COMMENCEMENT DATE", "VALUATION DATE"]:
            try:
                parsed_date = pd.to_datetime(value, errors='coerce')
                value = parsed_date if pd.notnull(parsed_date) else value
            except:
                pass
        
        # Convert to number if it's a numeric field but not a date or POLICY NUMBER
        if header_upper not in ["POLICY NUMBER", "DATE OF BIRTH", "COMMENCEMENT DATE", "VALUATION DATE"]:
            try:
                num = float(value)
                if not pd.isna(num):
                    value = num
            except:
                pass

       # === Format SPOUSE REVERSION RATE ===
if 'SPOUSE REVERSION RATE' in aligned_df.columns:
    def format_spouse(val):
        try:
            # Convert to float, then to integer
            num = float(val)
            return int(round(num))  # Return as integer for internal use
        except:
            return 0  # Default to 0 if the value is not valid
    aligned_df['SPOUSE REVERSION RATE'] = aligned_df['SPOUSE REVERSION RATE'].apply(format_spouse)

# === Append data to worksheet ===
for row in dataframe_to_rows(aligned_df, index=False, header=False):
    for col_idx, value in enumerate(row, start=1):
        header_cell = ws.cell(row=1, column=col_idx)
        header_upper = str(header_cell.value).strip().upper() if header_cell.value else ""
        
        # Handle dates correctly
        if header_upper in ["DATE OF BIRTH", "COMMENCEMENT DATE", "VALUATION DATE"]:
            try:
                parsed_date = pd.to_datetime(value, errors='coerce')
                value = parsed_date if pd.notnull(parsed_date) else value
            except:
                pass
        
        # Convert to number if it's a numeric field but not a date or POLICY NUMBER
        if header_upper not in ["POLICY NUMBER", "DATE OF BIRTH", "COMMENCEMENT DATE", "VALUATION DATE"]:
            try:
                num = float(value)
                if not pd.isna(num):
                    value = num
            except:
                pass

        # === Format SPOUSE REVERSION RATE as integer percentage ===
        if header_upper == "SPOUSE REVERSION RATE":
            # Store as integer and apply percentage formatting in Excel
            value = int(value) / 100  # Store as 0.5 for 50%
        
        cell = ws.cell(row=start_row, column=col_idx, value=value)
        
        if col_idx in column_formats:
            fmt = column_formats[col_idx]
            cell.font = fmt['font']
            cell.fill = fmt['fill']
            cell.border = fmt['border']
            cell.alignment = fmt['alignment']
            
            # Apply correct number formatting
            if header_upper in ["DATE OF BIRTH", "COMMENCEMENT DATE", "VALUATION DATE"]:
                cell.number_format = 'DD/MM/YYYY'
            elif header_upper != "POLICY NUMBER":
                cell.number_format = '#,##0'
            else:
                cell.number_format = fmt['number_format']

            # Apply percentage format for SPOUSE REVERSION RATE
            if header_upper == "SPOUSE REVERSION RATE":
                cell.number_format = '0%'  # Format as percentage (e.g., 50% instead of 0.5)

        if header_upper == "POLICY NUMBER":
            cell.fill = policy_fill
    start_row += 1



# === Save workbook ===
wb.move_sheet(backup_ws, offset=-1)
output_file = os.path.join(output_folder, 'Annuity valuation_updated.xlsx')
try:
    wb.save(output_file)
    print(f"✅ File saved successfully: {output_file}")
except Exception as e:
    print(f"❌ Error saving file: {e}")


📁 Enter the path to the main folder containing 'input' and 'output' subfolders:  D:\Model
📅 Enter the valuation date (DD/MM/YYYY):  31/03/2025


C:\Users\chris\AppData\Local\Temp\ipykernel_10504\2972249945.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  register_df['Valuation date'] = pd.to_datetime(register_df['Valuation date'], errors='coerce')


✅ File saved successfully: D:\Model\output\Annuity valuation_updated.xlsx
